In [ ]:
import json
import numpy as np
import pandas as pd
from haversine import haversine
import csv

In [ ]:
def clusterin(cluster_lat, cluster_lon, location_lat, location_lon, cluster_max,cluster_number):
  cluster_center = (cluster_lat, cluster_lon)
  location = (location_lat, location_lon)
  # 클러스터에 있는지 아닌지 확인해 주는 함수 (클러스터의 중심과, 현재 위치와의 거리를 계산해서 cluster 의 크기로 지정한 maxmean인지 확인) 
  #클러스터에 포함된다면 몇번 클러스터인지 반환
  if haversine(cluster_center, location) < cluster_max:
    return int(cluster_number)
  else :
    return -1

In [ ]:
# 리스트 파일이 있는 경로
list_dir = "./list/list_cdy"

In [ ]:
# 만들어진 클러스터의 정보 가져오기
with open("./cdy/results/integratedJSON/Integrated_cdy_Clustering_Result.json") as st_json:
    st_python = json.load(st_json)

In [ ]:
number =[]
lat =[]
lon =[]
max1 =[]

for clusters in st_python:
    number.append(clusters["cluster"])
    lat.append(clusters["latitude"])
    lon.append(clusters["longitude"])
    max1.append(clusters["maxDistance"])
    i += 1

In [ ]:
'''
cluster_info = pd.DataFrame({
    
            "cluster_number":number,
            "latitude":lat,
            "longitude":lon,
            "maxDistance":max1
})

print(cluster_info)
'''

In [ ]:
#numpy로 받아오는 방법 
#일단은 pandas로 해볼것 
cluster_info = np.zeros((100,5))
cluster_number = 0
for clusters in st_python:
    cluster_info[cluster_number][0]= int(clusters["cluster"])
    cluster_info[cluster_number][1] = clusters["latitude"]
    cluster_info[cluster_number][2] = clusters["longitude"]
    cluster_info[cluster_number][3] = clusters["maxDistance"]
    cluster_number += 1
    cluster_info.resize(cluster_number+1,4)

print(cluster_number)

In [ ]:
# list를 이용하여 위치 데이터 받아오기
list = open(list_dir)
list_line = list.read().split("\n")

del list_line[-1]
print(list_line)

In [ ]:
########### read the location data 
location = np.zeros((50,3))

k = 0
for i in list_line:
    trace = open(i).read().split()
    for j in range(0,len(trace)-10):
        if j % 5 == 2:   
            location[k][0] = np.int(trace[j][0:10])
            location[k][1] = np.float(trace[j+1])
            location[k][2] = np.float(trace[j+2])
            k = k+1
            location.resize(k+1,3)

location.resize(k,3)
print(location)

In [ ]:
for j in range(k):
    print(location[j][0])
    for i in range(cluster_number):
        now_cluster = clusterin(cluster_info[i][1],cluster_info[i][2], location[j][1],location[j][2], cluster_info[i][3],cluster_info[i][0])
        if now_cluster != -1:
            print('in cluster: ', now_cluster)
            break
    if now_cluster == -1:
        print('not in cluster')

In [ ]:
#이제부터 클러스터에 들어갔을때   , 나왔을때 시간을 적어줘보자 
# 초기값 
before_cluster = -1 
for j in range(k):
    #print(location[j][0], before_cluster)
    for i in range(cluster_number):
        now_cluster = clusterin(cluster_info[i][1],cluster_info[i][2], location[j][1],location[j][2], cluster_info[i][3],cluster_info[i][0])
        if now_cluster != -1: 
            if before_cluster == now_cluster:
                #print('in_cluster',now_cluster,location[j][0])
                break
            elif before_cluster == -1:
                print('in', now_cluster, location[j][0] )
                
            else :
                print('out', before_cluster ,location[j][0])
                print('in', now_cluster, location[j][0]  )
                
            before_cluster = now_cluster
            break
        else:
            continue
    if now_cluster == -1 and before_cluster != -1:
        print('out', before_cluster, location[j][0] )
        before_cluster = now_cluster
    
                   

In [ ]:
time = []
cluster_num_info = []
cluster_inout =[]  

before_cluster = -1 
for j in range(k):
    for i in range(cluster_number):
        now_cluster = clusterin(cluster_info[i][1],cluster_info[i][2], location[j][1],location[j][2], cluster_info[i][3],cluster_info[i][0])
        if now_cluster != -1: 
            if before_cluster == now_cluster:
                break
            elif before_cluster == -1:
                time.append(location[j][0])
                cluster_num_info.append(now_cluster)
                cluster_inout.append('in')
            else :
                time.append(location[j][0])
                cluster_num_info.append(before_cluster)
                cluster_inout.append('out')
                time.append(location[j][0])
                cluster_num_info.append(now_cluster)
                cluster_inout.append('in')
            before_cluster = now_cluster
            break
        else:
            continue
    if now_cluster == -1 and before_cluster != -1:
        time.append(location[j][0])
        cluster_num_info.append(before_cluster)
        cluster_inout.append('out')
        before_cluster = now_cluster


In [ ]:
cluster_visit = pd.DataFrame({
            
            "time":time,
            "cluster":cluster_num_info,
            "in_out":cluster_inout
            
})
print(cluster_visit)

In [ ]:
cluster_visit.to_csv('cluster_visit.csv', index=False, encoding='cp949')